In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

In [2]:
# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

In [3]:
# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

In [4]:
#examine the data
# school_data.head()

In [5]:
#examine the data
# student_data.head()

In [6]:
# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


In [7]:
# assign summary data to variables
schools_total = school_data_complete["School ID"].nunique()
student_total = school_data_complete["Student ID"].nunique()
total_budget = school_data_complete["budget"].unique().sum()
average_math = school_data_complete["math_score"].mean(axis=0)
average_reading = school_data_complete["reading_score"].mean(axis=0)
passing_math_count = school_data_complete.math_score[(school_data_complete.math_score  >= 70)].count()
passing_math_percent = passing_math_count / student_total * 100
passing_reading_count = school_data_complete.reading_score[(school_data_complete.reading_score >= 70)].count()
passing_reading_percent = passing_reading_count / student_total * 100 
passing_both_count = len(school_data_complete[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70) ])
passing_both_percent = (passing_both_count / student_total) * 100


In [8]:
# create summary dataframe
combined_df = pd.DataFrame({"Total Schools": [schools_total],
                            "Total Students": [student_total],
                            "Total Budget":   [total_budget], # is this value correct?
                            "Average Math Score": [f'{round(average_math,1)}%'],
                            "Average Reading Score": [f'{round(average_reading, 1)}%'],
                            "Students Passing Math": [f'{round(passing_math_percent, 1)}%'],
                            "Students Passing Reading": [f'{round(passing_reading_percent, 1)}%'],
                            "Overall Passing": [f'{round(passing_both_percent, 1)}%']
                           
                           })
# formatting
combined_df['Total Budget'] = combined_df['Total Budget'].astype(int).apply(lambda x: "${:,}".format(x))
combined_df['Total Students'] = combined_df['Total Students'].astype(int).apply(lambda x: "{:,}".format(x))

# show df
combined_df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading,Overall Passing
0,15,"39,170","$24,649,428",79.0%,81.9%,75.0%,85.8%,65.2%


In [9]:
# Group by schools
groupby_school = school_data_complete.groupby(['school_name'])



# Create series for df
school_type = groupby_school['type'].unique()
size = groupby_school['size'].unique()
budget_per_school = groupby_school['budget'].mean()
average_math_per_school = groupby_school['math_score'].mean() 
average_reading_per_school = groupby_school['reading_score'].mean()
passing_math = school_data_complete.loc[school_data_complete['math_score'] >= 70, ['school_name','math_score']]
passing_math_counts = passing_math['school_name'].value_counts()
passing_reading = school_data_complete.loc[school_data_complete['reading_score'] >= 70, ['school_name','reading_score']]
passing_reading_counts = passing_reading['school_name'].value_counts()
passing_both = school_data_complete.loc[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70), ['school_name','math_score']]
passing_both_counts = passing_both['school_name'].value_counts()
passing_math_counts = passing_math_counts.sort_index()
passing_reading_counts = passing_reading_counts.sort_index()
passing_both_counts = passing_both_counts.sort_index()
print(passing_both_counts)

Bailey High School       2719
Cabrera High School      1697
Figueroa High School     1569
Ford High School         1487
Griffin High School      1330
Hernandez High School    2481
Holden High School        381
Huang High School        1561
Johnson High School      2549
Pena High School          871
Rodriguez High School    2119
Shelton High School      1583
Thomas High School       1487
Wilson High School       2068
Wright High School       1626
Name: school_name, dtype: int64


In [10]:
schools_summary_df = pd.concat([school_type, size, budget_per_school, average_math_per_school, average_reading_per_school], axis = 1)

In [11]:
schools_summary_df['Passing Math Percent'] = round(((passing_math_counts / size) * 100).astype(float), 1)

In [12]:
schools_summary_df['Passing Reading Percent'] = round(((passing_reading_counts / size) * 100).astype(float), 1)

In [42]:
schools_summary_df['Passing Both Percent'] = round(((passing_both_counts / size) * 100).astype(float), 1)

schools_summary_df['type'] = schools_summary_df['type'].str[0]
schools_summary_df['size'] = schools_summary_df['size'].astype(int)
schools_summary_df['Budget per student'] = schools_summary_df['budget'] / schools_summary_df['size']
# schools_summary_df

In [41]:
# new dfs sorted by passing %
top_five = schools_summary_df.sort_values('Passing Both Percent', ascending = False)
# top_five.head()

In [40]:
# new df sorted be lowest passing #
bottom_five = schools_summary_df.sort_values('Passing Both Percent', ascending = True)
# bottom_five.head()

In [22]:
# series sorted by grade (math)
ninth_grade_math = school_data_complete.loc[school_data_complete['grade'] == "9th", ['school_name', 'math_score']]
tenth_grade_math = school_data_complete.loc[school_data_complete['grade'] == "10th", ['school_name', 'math_score']]
eleventh_grade_math = school_data_complete.loc[school_data_complete['grade'] == "11th", ['school_name', 'math_score']]
twelfth_grade_math = school_data_complete.loc[school_data_complete['grade'] == "12th", ['school_name', 'math_score']]

In [23]:
# group series by school name (math)
ninth_grade_math = ninth_grade_math.groupby('school_name').mean()
tenth_grade_math = tenth_grade_math.groupby('school_name').mean()
eleventh_grade_math = eleventh_grade_math.groupby('school_name').mean()
twelfth_grade_math = twelfth_grade_math.groupby('school_name').mean()


In [39]:
# combine series in to df (math)

math_score_by_grade_df = pd.DataFrame({'9th': ninth_grade_math['math_score'],
                                      '10th': tenth_grade_math['math_score'],
                                       '11th': eleventh_grade_math['math_score'],
                                       '12th': twelfth_grade_math['math_score']
                                      })

# math_score_by_grade_df

In [31]:
# series sorted by grade (reading)
ninth_grade_reading = school_data_complete.loc[school_data_complete['grade'] == "9th", ['school_name', 'reading_score']]
tenth_grade_reading = school_data_complete.loc[school_data_complete['grade'] == "10th", ['school_name', 'reading_score']]
eleventh_grade_reading = school_data_complete.loc[school_data_complete['grade'] == "11th", ['school_name', 'reading_score']]
twelfth_grade_reading = school_data_complete.loc[school_data_complete['grade'] == "12th", ['school_name', 'reading_score']]

In [33]:
# group series by school name (reading)
ninth_grade_reading = ninth_grade_reading.groupby('school_name').mean()
tenth_grade_reading = tenth_grade_reading.groupby('school_name').mean()
eleventh_grade_reading = eleventh_grade_reading.groupby('school_name').mean()
twelfth_grade_reading = twelfth_grade_reading.groupby('school_name').mean()

In [38]:
# combine series in to df (reading)

reading_score_by_grade_df = pd.DataFrame({'9th': ninth_grade_reading['reading_score'],
                                      '10th': tenth_grade_reading['reading_score'],
                                       '11th': eleventh_grade_reading['reading_score'],
                                       '12th': twelfth_grade_reading['reading_score']
                                      })

# reading_score_by_grade_df

In [44]:
# create bins
bins = [0, 585, 630, 645, 680]
my_labels = ['<$585', '$585-630', '$630-645', '$645-680']

# cut together
schools_summary_df['Spend Range (per student)'] = pd.cut(schools_summary_df['Budget per student'], bins, labels=my_labels)



In [48]:
# groupby budget group
budget_bins_df = schools_summary_df.groupby('Spend Range (per student)').mean()
budget_bins_df = budget_bins_df[['math_score','reading_score','Passing Math Percent','Passing Reading Percent', 'Passing Both Percent']]
budget_bins_df

,math_score,reading_score,Passing Math Percent,Passing Reading Percent,Passing Both Percent
Spend Range (per student),,,,,
<$585,83.455399,83.933814,93.45,96.600000,90.35
$585-630,81.899826,83.155286,87.15,92.700000,81.40
$630-645,78.518855,81.624473,73.50,84.375000,62.85
$645-680,76.997210,81.027843,66.20,81.133333,53.50


In [52]:
# create bins for school size
bins2 = [0, 1000, 2000, 5000]
my_labels2 = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']

# cut together
size_bins_df = schools_summary_df['School Size'] = pd.cut(schools_summary_df['size'], bins2, labels=my_labels2)

size_bins_df = schools_summary_df.groupby('School Size').mean()
size_bins_df = size_bins_df[['math_score','reading_score','Passing Math Percent','Passing Reading Percent', 'Passing Both Percent']]
size_bins_df

,math_score,reading_score,Passing Math Percent,Passing Reading Percent,Passing Both Percent
School Size,,,,,
Small (<1000),83.821598,83.929843,93.5500,96.10,89.850
Medium (1000-2000),83.374684,83.864438,93.6000,96.78,90.600
Large (2000-5000),77.746417,81.344493,69.9875,82.75,58.275


In [54]:
school_type_df = schools_summary_df.groupby('type').mean()
school_type_df = school_type_df['ma']

,size,budget,math_score,reading_score,Passing Math Percent,Passing Reading Percent,Passing Both Percent,Budget per student
type,,,,,,,,
C,1524.250000,9.126881e+05,83.473852,83.896421,93.625000,96.575000,90.412500,599.500000
D,3853.714286,2.478275e+06,76.956733,80.966636,66.571429,80.785714,53.657143,643.571429
